In [8]:
###############################
# Relative Trust-Region ICA
# (Dogleg on the relative model)
###############################
using LinearAlgebra, Statistics, Random

# -----------------------------
# Whitening (PCA-whiten the data)
# X: m×n (m dims, n samples). Returns:
#   Xw     : whitened data (m×n)
#   Wwhite : whitening matrix so that Xw = Wwhite * (X - mean)
# -----------------------------
function whiten(X::AbstractMatrix{T}) where {T<:AbstractFloat}
    m, n = size(X)
    μ  = mean(X; dims=2)
    Xc = X .- μ
    S  = (Xc*Xc')/n
    λ, V = eigen(Symmetric(S))
    Dm12 = Diagonal((λ .+ eps(T)).^(-one(T)/2))
    Wwhite = Dm12 * V'
    return Wwhite * Xc, Wwhite
end

# -----------------------------
# Log-likelihood (to maximize)
# Stable identity: log(2cosh y) = |y| + log1p(exp(-2|y|))
# f(W) = log|det W| - (1/n) * sum_{i,t} log(2cosh(Y_it)), Y = W*Xw
# -----------------------------
function ica_loss(W::AbstractMatrix{T}, Y::AbstractMatrix{T}) where {T<:AbstractFloat}
    n = size(Y,2)
    A = abs.(Y)
    s = sum(A) + sum(log1p.(exp.(-T(2).*A)))
    log_abs_det, _ = logabsdet(W)
    return log_abs_det - s/n
end

# -----------------------------
# Score and its derivative for log-cosh model:
# ψ(y)   = tanh(y/2)
# ψ'(y)  = 0.5 * sech^2(y/2) = 0.5 / cosh(y/2)^2
# Matrix versions (elementwise)
# -----------------------------
# psi_mat(Y::AbstractMatrix{T}) where {T<:AbstractFloat} = tanh.(Y ./ T(2))
# function psid_mat(Y::AbstractMatrix{T}) where {T<:AbstractFloat}
#     U = Y ./ T(2)
#     C = cosh.(U)
#     (one(T)/2) .* (inv.(C).^2)
# end
# # score & curvature consistent with log(2cosh y)
psi_mat(Y::AbstractMatrix{T}) where {T<:AbstractFloat}  = tanh.(Y)          # ← was tanh(Y/2)
function psid_mat(Y::AbstractMatrix{T}) where {T<:AbstractFloat}
    C = cosh.(Y)
    inv.(C).^2                                                               # ← was 0.5 * inv(cosh(Y/2))^2
end

# -----------------------------
# Relative gradient:
# G = (1/n) * ψ(Y) * Y' - I
# -----------------------------
# function relgrad(Y::AbstractMatrix{T}) where {T<:AbstractFloat}
#     n = size(Y,2)
#     psi_mat(Y) * Y' / n .- I(size(Y,1))
# end
function relgrad(Y::AbstractMatrix{T}) where {T<:AbstractFloat}
    n = size(Y,2)
    I(size(Y,1)) .- psi_mat(Y) * Y' / n
end

# -----------------------------
# Row-block (diagonal-by-row) relative Hessian approximation:
# For row i:
#   H_i = (1/n) * (Y .* w_i) * Y' + e_i e_i^T
# where w_i = ψ'(Y[i,:]) (1×n) and ".*" is column weighting.
# Returns a vector of Symmetric blocks {H_1,...,H_m}.
# -----------------------------
function hess_blocks(Y::AbstractMatrix{T}; ridge::T=T(1e-6)) where {T<:AbstractFloat}
    m, n = size(Y)
    Wgt = psid_mat(Y)
    H = Vector{Symmetric{T,Matrix{T}}}(undef, m)
    for i in 1:m
        wi = view(Wgt, i, :)
        HW = Y .* reshape(wi, 1, :)
        Hi = (HW * Y') / n
#         Hi[i,i] += one(T)
        Hi += I
#         @inbounds for j in 1:m
#             Hi[j,j] += ridge            # small damping
#         end
        H[i] = Symmetric(Hi)
    end
    return H
end

# -----------------------------
# Apply block-diagonal Hessian to a direction matrix P (m×m):
# R = H̃ * P, where each row i: R[i,:] = H_i * P[i,:]^T (then transposed to row)
# -----------------------------
function hvp_block(H::Vector{Symmetric{T,Matrix{T}}}, P::AbstractMatrix{T}) where {T<:AbstractFloat}
    m = length(H)
    R = similar(P)                     # m×m
    @inbounds for i in 1:m
        Hi = Matrix(H[i])              # m×m
        v  = (@view P[i, :])[:]        # m-vector (row → Vector)
        r  = Hi * v                    # m-vector
        @views copyto!(R[i, :], r')    # write back as a row (no broadcasting pitfalls)
    end
    return R
end

# Frobenius inner product and norm
fro_dot(A,B) = sum(A .* B)
fro_norm(A) = sqrt(fro_dot(A,A))

# -----------------------------
# Dogleg step (trust-region subproblem) for MAXIMIZATION:
# Model: m(E) = f + <G,E> - 1/2 <E, H̃ E>
# Cauchy step along +G, “Newton” step solves H_i e_i = G_i, both clipped to ||E||_F ≤ Δ
# Returns:
#   p    : step E (m×m)
#   pred : predicted increase (≥ 0 ideally)
# -----------------------------
function dogleg_step(G::AbstractMatrix{T}, H::Vector{Symmetric{T,Matrix{T}}}, Δ::T) where {T<:AbstractFloat}
    # Cauchy (steepest-ascent) step: pU =  α G,   α = ||G||^2 / <G, H̃ G>
    HG  = hvp_block(H, G)
    gHg = fro_dot(G, HG)
    α   = gHg ≤ 0 ? (Δ / max(fro_norm(G), eps(T))) : (fro_dot(G,G) / gHg)
    pU  = α .* G
    nU  = fro_norm(pU)

    # Block-Newton step pB: solve H_i e_i = G_i row-by-row
    m = size(G,1)
    pB = zeros(T, size(G))
    for i in 1:m
        Hi = Matrix(H[i])
        gi = (@view G[i, :])[:]          # Vector
        ei =  (Hi \ gi)                  # ascent “Newton” step (note: +, not -)
        pB[i, :] = ei'
    end
    nB = fro_norm(pB)

    # Choose dogleg step within trust radius Δ (maximization variant)
    if nB ≤ Δ
        p = pB
    elseif nU ≥ Δ
        # Clip the Cauchy step along +G to the boundary
        p =  (Δ / max(fro_norm(G), eps(T))) .* G
    else
        # Interpolate between pU and pB to hit the boundary: ||pU + τ(pB-pU)|| = Δ
        d = pB .- pU
        a = fro_dot(d,d)
        b = T(2) * fro_dot(pU, d)
        c = fro_dot(pU,pU) - Δ^2
        τ = (-b + sqrt(max(b^2 - 4a*c, zero(T)))) / (T(2)*a)
        p = pU .+ τ .* d
    end

    # Predicted increase (for maximization): pred = <G,p> - 1/2 <p, H̃ p>
    Hp   = hvp_block(H, p)
    pred = fro_dot(G, p) - T(0.5) * fro_dot(p, Hp)
    pred = max(pred, zero(T))            # numerical safeguard
    return p, pred
end

# -----------------------------
# One trust-region iteration (updates W,Y if accepted)
# W_new = (I + E) W
# Returns:
#   Δ     : updated radius
#   ρ     : ratio ared/pred
#   accept: whether step accepted
#   ftry  : trial objective
# -----------------------------
function trust_region_step!(W::AbstractMatrix{T}, Y::AbstractMatrix{T}, X::AbstractMatrix{T},
                            Δ::T; η::T=T(0.1), Δmax::T=T(1e2)) where {T<:AbstractFloat}
    f0 = ica_loss(W, Y)
    G  = relgrad(Y)
    H  = hess_blocks(Y)

    # Dogleg proposal
    E, pred = dogleg_step(G, H, Δ)
    Iplus  = Matrix{T}(I, size(W,1), size(W,2))
    Wtrial = (Iplus .+ E) * W
    Ytrial = Wtrial * X
    ftrial = ica_loss(Wtrial, Ytrial)

    ared = ftrial - f0
    ρ    = pred == 0 ? T(0) : (ared / pred)

    accept = false
    if ρ < T(0.25)
        Δ = max(Δ*T(0.25), T(1e-8))
        # ---- SAFEGUARD: fallback Armijo along +G when Δ is tiny or repeated rejects ----
        if Δ ≤ T(1e-6)
            # take a gradient ascent step: W ← (I + α G) W with backtracking
            α  = T(1.0)
            c1 = T(1e-4)
            GE = fro_dot(G, G)                     # ⟨G, G⟩ ≥ 0  (since direction is +G)
            while α > T(1e-12)
                Wls = (Iplus .+ α .* G) * W
                Yls = Wls * X
                fls = ica_loss(Wls, Yls)
                if fls ≥ f0 + c1 * α * GE          # Armijo condition for maximization
                    W .= Wls
                    Y .= Yls
                    accept = true
                    # modestly increase Δ so TR can recover
                    Δ = max(Δ, T(1e-3))
                    ftrial = fls
                    break
                end
                α *= T(0.5)
            end
        end
    else
        if ρ > T(0.75) && abs(fro_norm(E) - Δ) ≤ T(1e-7)
            Δ = min(T(2)*Δ, Δmax)
        end
        if ρ > η
            W .= Wtrial
            Y .= Ytrial
            accept = true
        end
    end

    return Δ, ρ, accept, ftrial
end

# -----------------------------
# Main solver (relative trust-region with dogleg)
# Returns:
#   W  : unmixing in whitened space
#   S  : overall unmixing on raw data (W * Wwhite)
#   Y  : sources in whitened space (W * Xw)
#   hist: objective history
# -----------------------------
function ica_relative_trustregion(X::AbstractMatrix{R};
                                  maxiter::Int=200, tol::Float64=1e-6,
                                  Δ0::Float64=1.0, η::Float64=0.1) where {R<:Real}
    X = Matrix{Float64}(X)
    T = eltype(X)
    m, n = size(X)
    @assert n ≥ m "Need n ≥ m samples."

    # Whitening
    Xw, Wwhite = whiten(X)

    # Initialize
    W = Matrix{T}(I, m, m)
    Y = W * Xw
    hist = Float64[]
    Δ = T(Δ0)

    for it in 1:maxiter
        
        if it == 1
            G = relgrad(Y)
            ε = 1e-3
            Wt = (I + ε*G) * W
            Yt = Wt * Xw
            @show :fd_check  ica_loss(Wt, Yt) - ica_loss(W, Y)   # 应该 > 0
        end
        
        push!(hist, ica_loss(W, Y))
        gnorm = opnorm(relgrad(Y), Inf)   # stop by gradient infinity norm
        # (Optional) monitor progress:
        @info "iter=$it" f=hist[end] g∞=gnorm Δ=Δ

        if gnorm < tol
            break
        end

        Δ, ρ, acc, ftrial = trust_region_step!(W, Y, Xw, Δ; η=T(η))
        # (Optional) print a short log
        # @info "TR" ρ=ρ acc=acc Δ=Δ ftry=ftrial
    end

    S = W * Wwhite
    return W, S, Y, hist
end



ica_relative_trustregion (generic function with 1 method)

In [9]:
W, Sunmix, Yest, hist = ica_relative_trustregion(X; maxiter=200, tol=1e-6, Δ0=5.0, η=0.05)

:fd_check = :fd_check

┌ Info: iter=1
│   f = -5.338781718512037
│   g∞ = 0.40488422148709907
└   Δ = 5.0
┌ Info: iter=2
│   f = -4.973397346643575
│   g∞ = 0.135848619395037
└   Δ = 5.0
┌ Info: iter=3
│   f = -4.941313344319518
│   g∞ = 0.028242256520308116
└   Δ = 5.0
┌ Info: iter=4
│   f = -4.940891893780482
│   g∞ = 0.01674665973155644
└   Δ = 5.0
┌ Info: iter=5
│   f = -4.940762776608576
│   g∞ = 0.016670073222110317
└   Δ = 5.0
┌ Info: iter=6
│   f = -4.940632914867923
│   g∞ = 0.016705949818271035
└   Δ = 5.0
┌ Info: iter=7
│   f = -4.940502273404396
│   g∞ = 0.0167429554014047
└   Δ = 5.0
┌ Info: iter=8
│   f = -4.940370837038227
│   g∞ = 0.01678106921309515
└   Δ = 5.0
┌ Info: iter=9
│   f = -4.940238590855386
│   g∞ = 0.016820266295935936
└   Δ = 5.0
┌ Info: iter=10
│   f = -4.94010552028771
│   g∞ = 0.01686051692086163
└   Δ = 5.0
┌ Info: iter=11
│   f = -4.939971611196414
│   g∞ = 0.016901786428182488
└   Δ = 5.0
┌ Info: iter=12
│   f = -4.93983684995978
│   g∞ = 0.01694403511837485
└   Δ = 5.0
┌


ica_loss(Wt, Yt) - ica_loss(W, Y) = 0.0007767418647617319


┌ Info: iter=55
│   f = -4.933287663249294
│   g∞ = 0.018289070478818715
└   Δ = 5.0
┌ Info: iter=56
│   f = -4.933127826291355
│   g∞ = 0.018263629726579012
└   Δ = 5.0
┌ Info: iter=57
│   f = -4.9329685402603385
│   g∞ = 0.018233585316204625
└   Δ = 5.0
┌ Info: iter=58
│   f = -4.932809884738272
│   g∞ = 0.018198870941808498
└   Δ = 5.0
┌ Info: iter=59
│   f = -4.932651939099473
│   g∞ = 0.01815943218383271
└   Δ = 5.0
┌ Info: iter=60
│   f = -4.932494782219452
│   g∞ = 0.01811522685022996
└   Δ = 5.0
┌ Info: iter=61
│   f = -4.932338492186103
│   g∞ = 0.01806622523590485
└   Δ = 5.0
┌ Info: iter=62
│   f = -4.932183146015764
│   g∞ = 0.018012410296532423
└   Δ = 5.0
┌ Info: iter=63
│   f = -4.932028819376624
│   g∞ = 0.01795377773404702
└   Δ = 5.0
┌ Info: iter=64
│   f = -4.9318755863219
│   g∞ = 0.01789033599234599
└   Δ = 5.0
┌ Info: iter=65
│   f = -4.931723519035019
│   g∞ = 0.01782210616301695
└   Δ = 5.0
┌ Info: iter=66
│   f = -4.931572687588936
│   g∞ = 0.01774912180225523


┌ Info: iter=121
│   f = -4.926005714415498
│   g∞ = 0.009895721119178262
└   Δ = 5.0
┌ Info: iter=122
│   f = -4.92595145887789
│   g∞ = 0.009747658508271077
└   Δ = 5.0
┌ Info: iter=123
│   f = -4.925898446543001
│   g∞ = 0.009600967835896871
└   Δ = 5.0
┌ Info: iter=124
│   f = -4.9258466498773625
│   g∞ = 0.009455685392683063
└   Δ = 5.0
┌ Info: iter=125
│   f = -4.92579604157428
│   g∞ = 0.009311844958070411
└   Δ = 5.0
┌ Info: iter=126
│   f = -4.925746594574297
│   g∞ = 0.009169477871180226
└   Δ = 5.0
┌ Info: iter=127
│   f = -4.925698282084074
│   g∞ = 0.009028613100745469
└   Δ = 5.0
┌ Info: iter=128
│   f = -4.925651077593804
│   g∞ = 0.008889277314132861
└   Δ = 5.0
┌ Info: iter=129
│   f = -4.9256049548932115
│   g∞ = 0.008751494945480687
└   Δ = 5.0
┌ Info: iter=130
│   f = -4.925559888086257
│   g∞ = 0.008615288263005368
└   Δ = 5.0
┌ Info: iter=131
│   f = -4.925515851604579
│   g∞ = 0.008480677435491576
└   Δ = 5.0
┌ Info: iter=132
│   f = -4.925472820219744
│   g∞ = 0

┌ Info: iter=187
│   f = -4.924081259365605
│   g∞ = 0.004509715823575425
└   Δ = 5.0
┌ Info: iter=188
│   f = -4.924066193809351
│   g∞ = 0.004502955489197556
└   Δ = 5.0
┌ Info: iter=189
│   f = -4.924051316908053
│   g∞ = 0.004496133682771167
└   Δ = 5.0
┌ Info: iter=190
│   f = -4.924036623166836
│   g∞ = 0.004489253787794446
└   Δ = 5.0
┌ Info: iter=191
│   f = -4.92402210728429
│   g∞ = 0.004482318964473074
└   Δ = 5.0
┌ Info: iter=192
│   f = -4.924007764147053
│   g∞ = 0.0044753321574601546
└   Δ = 5.0
┌ Info: iter=193
│   f = -4.923993588824512
│   g∞ = 0.004468296103494637
└   Δ = 5.0
┌ Info: iter=194
│   f = -4.923979576563582
│   g∞ = 0.004461213338955463
└   Δ = 5.0
┌ Info: iter=195
│   f = -4.923965722783592
│   g∞ = 0.0044540862073034725
└   Δ = 5.0
┌ Info: iter=196
│   f = -4.923952023071256
│   g∞ = 0.0044469168664212355
└   Δ = 5.0
┌ Info: iter=197
│   f = -4.92393847317576
│   g∞ = 0.004439707295830482
└   Δ = 5.0
┌ Info: iter=198
│   f = -4.923925069003927
│   g∞ = 

([1.2346527736626511 0.11660786078999517 … 0.3499903363237628 -0.13354976253249037; -0.26384602016386755 1.419941574992384 … 0.20964689807764694 0.07104142541600028; … ; -0.1786207797293473 -0.16087886868042936 … 1.2776137361981772 -0.5375630808951558; -0.1875667717290092 -0.21973529997029462 … 0.5851820210529712 1.2812254508899223], [23.535680212485726 -89.67318778460003 … -43.34561311730067 22.63295829690875; -5.274295281099494 17.924758555956274 … 10.98824230427037 -6.155284371411015; … ; -3.3562935636641247 13.083095966758965 … 6.239070897778853 -2.948069882272659; -3.6011284836009905 13.809659822412979 … 6.34617391383202 -3.2744366411436396], [2.2232469292385497 -0.766193599417572 … 0.3690688809543941 0.6192389326044461; 1.4598922130469711 -0.2823735145668684 … 1.9247650476634257 1.311015513594156; … ; 1.8917034287970993 -0.3225477467386677 … -2.2927488949216395 0.15123273278137325; -0.06993319025420805 2.75896234704053 … -0.541358860922898 -2.086161916010081], [-5.338781718512037

In [4]:
W, Sunmix, Yest, hist = ica_relative_trustregion(X; maxiter=200, tol=1e-6, Δ0=5.0, η=0.05)

:fd_check = :fd_check

┌ Info: iter=1
│   f = -5.338781718512037
│   g∞ = 0.40488422148709907
└   Δ = 5.0
┌ Info: iter=2
│   f = -4.973281233568723
│   g∞ = 0.13966577513486542
└   Δ = 5.0
┌ Info: iter=3
│   f = -4.941062215946225
│   g∞ = 0.027220244911539883
└   Δ = 5.0
┌ Info: iter=4
│   f = -4.940508116289986
│   g∞ = 0.019440401449200305
└   Δ = 5.0
┌ Info: iter=5
│   f = -4.940244192189077
│   g∞ = 0.01541226661058702
└   Δ = 5.0
┌ Info: iter=6
│   f = -4.939977010140654
│   g∞ = 0.019235369183601785
└   Δ = 5.0
┌ Info: iter=7
│   f = -4.939706320385991
│   g∞ = 0.015400984856231087
└   Δ = 5.0
┌ Info: iter=8
│   f = -4.93943213898044
│   g∞ = 0.019128743131324465
└   Δ = 5.0
┌ Info: iter=9
│   f = -4.939154258498672
│   g∞ = 0.01566308409187222
└   Δ = 5.0
┌ Info: iter=10
│   f = -4.938872730559815
│   g∞ = 0.019088455570113984
└   Δ = 5.0
┌ Info: iter=11
│   f = -4.9385873802635745
│   g∞ = 0.016293567757773752
└   Δ = 5.0
┌ Info: iter=12
│   f = -4.938298308998762
│   g∞ = 0.019454333608698908
└   Δ


ica_loss(Wt, Yt) - ica_loss(W, Y) = 0.0007767418647617319


┌ Info: iter=55
│   f = -4.926776079395555
│   g∞ = 0.018665149941100384
└   Δ = 5.0
┌ Info: iter=56
│   f = -4.926635169491279
│   g∞ = 0.014388206242463553
└   Δ = 5.0
┌ Info: iter=57
│   f = -4.9265004460380855
│   g∞ = 0.018181648898264832
└   Δ = 5.0
┌ Info: iter=58
│   f = -4.926371640117699
│   g∞ = 0.013958642663019612
└   Δ = 5.0
┌ Info: iter=59
│   f = -4.926248573313185
│   g∞ = 0.017677327598015064
└   Δ = 5.0
┌ Info: iter=60
│   f = -4.926130973392358
│   g∞ = 0.013518784384143142
└   Δ = 5.0
┌ Info: iter=61
│   f = -4.926018664554693
│   g∞ = 0.017156321522186907
└   Δ = 5.0
┌ Info: iter=62
│   f = -4.925911377076949
│   g∞ = 0.013122979955186206
└   Δ = 5.0
┌ Info: iter=63
│   f = -4.925808940335463
│   g∞ = 0.01662274528749622
└   Δ = 5.0
┌ Info: iter=64
│   f = -4.925711091646453
│   g∞ = 0.012998842715563048
└   Δ = 5.0
┌ Info: iter=65
│   f = -4.925617667297094
│   g∞ = 0.016080642023055768
└   Δ = 5.0
┌ Info: iter=66
│   f = -4.925528415115513
│   g∞ = 0.01286692496

┌ Info: iter=109
│   f = -4.9236947450625035
│   g∞ = 0.0067667668590095644
└   Δ = 5.0
┌ Info: iter=110
│   f = -4.923672378576723
│   g∞ = 0.00905821528974787
└   Δ = 5.0
┌ Info: iter=111
│   f = -4.923650359096444
│   g∞ = 0.006524076746824714
└   Δ = 5.0
┌ Info: iter=112
│   f = -4.923628689821993
│   g∞ = 0.008878607369941654
└   Δ = 5.0
┌ Info: iter=113
│   f = -4.923607343598636
│   g∞ = 0.006294297693039629
└   Δ = 5.0
┌ Info: iter=114
│   f = -4.923586325962096
│   g∞ = 0.008699769172749314
└   Δ = 5.0
┌ Info: iter=115
│   f = -4.923565611488394
│   g∞ = 0.006076737337405011
└   Δ = 5.0
┌ Info: iter=116
│   f = -4.923545207603848
│   g∞ = 0.008521766747458871
└   Δ = 5.0
┌ Info: iter=117
│   f = -4.923525090469874
│   g∞ = 0.005870715384258641
└   Δ = 5.0
┌ Info: iter=118
│   f = -4.923505269008379
│   g∞ = 0.008344667553841887
└   Δ = 5.0
┌ Info: iter=119
│   f = -4.923485720834812
│   g∞ = 0.005675568303723732
└   Δ = 5.0
┌ Info: iter=120
│   f = -4.923466456017775
│   g∞ = 

┌ Info: iter=173
│   f = -4.922774306042187
│   g∞ = 0.00328882505696418
└   Δ = 5.0
┌ Info: iter=174
│   f = -4.9227665827827405
│   g∞ = 0.004089148386512995
└   Δ = 5.0
┌ Info: iter=175
│   f = -4.922759017620049
│   g∞ = 0.003211603341666991
└   Δ = 5.0
┌ Info: iter=176
│   f = -4.922751611547575
│   g∞ = 0.003972541785231809
└   Δ = 5.0
┌ Info: iter=177
│   f = -4.922744358807082
│   g∞ = 0.0031350730372003022
└   Δ = 5.0
┌ Info: iter=178
│   f = -4.922737260125444
│   g∞ = 0.0038587064354013356
└   Δ = 5.0
┌ Info: iter=179
│   f = -4.922730309989855
│   g∞ = 0.003059328070454669
└   Δ = 5.0
┌ Info: iter=180
│   f = -4.922723508886717
│   g∞ = 0.0037476295240160334
└   Δ = 5.0
┌ Info: iter=181
│   f = -4.922716851540876
│   g∞ = 0.002984452543562111
└   Δ = 5.0
┌ Info: iter=182
│   f = -4.922710338222941
│   g∞ = 0.0036392934465682243
└   Δ = 5.0
┌ Info: iter=183
│   f = -4.922703963887768
│   g∞ = 0.0029105213054759346
└   Δ = 5.0
┌ Info: iter=184
│   f = -4.9226977286133575
│   

([1.1778776906253952 0.2064780480872055 … 0.37445963180806985 -0.1670956974003086; -0.3890331185919243 1.3388536342360289 … 0.4253903138095477 0.15376758256099118; … ; -0.17456492091280304 -0.3655228919716827 … 1.2328072846157878 -0.5754761688111318; -0.2257366749917234 -0.4030166117835581 … 0.5487708625457347 1.2045130878285164], [22.45218988305482 -85.62668657170273 … -41.239112723453324 21.494939824557846; -7.654524780268387 27.075412864486267 … 15.320677361155788 -8.36013965330798; … ; -3.229949077329256 12.969016062047059 … 5.848633425956825 -2.7007166802224987; -4.260780268472844 16.7485677471372 … 7.464875226637181 -3.845399758584186], [2.449808239928309 -0.7467734132047663 … 0.4334462064731491 0.6373970846591428; 1.5120065439184107 0.038145056062556616 … 1.4876271082590786 1.0387910829348392; … ; 1.7756557721537405 -0.23955512843980464 … -2.5854187160961164 -0.10669231237233459; 0.057614552827183564 2.920907158173786 … -0.8548805515660401 -2.4469317269239266], [-5.3387817185120

In [2]:

Random.seed!(0)
m, n = 5, 5000
# Create non-Gaussian sources and mix
Strue = reduce(hcat, [randn(n) .* (rand() < 0.5 ? 1.0 : 3.0) for _ in 1:m])'
Atrue = randn(m, m)
X = Atrue * Strue

W, Sunmix, Yest, hist = ica_relative_trustregion(X; maxiter=200, tol=1e-6,
                                                  Δ0=1.0, η=0.1)
println("Done. Final objective = ", last(hist))

┌ Info: iter=1
│   f = -5.338781718512037
│   g∞ = 0.5900719076374409
└   Δ = 1.0
┌ Info: iter=2
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 1.0
┌ Info: iter=3
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 0.25
┌ Info: iter=4
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 0.0625
┌ Info: iter=5
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 0.015625
┌ Info: iter=6
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 0.00390625
┌ Info: iter=7
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 0.0009765625
┌ Info: iter=8
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 0.000244140625
┌ Info: iter=9
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 6.103515625e-5
┌ Info: iter=10
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 1.52587890625e-5
┌ Info: iter=11
│   f = -4.941518123974778
│   g∞ = 0.2554623606999647
└   Δ = 3.814697265625e-6
┌ Info: iter=12
│   f 

┌ Info: iter=47
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=48
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=49
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=50
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=51
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=52
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=53
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=54
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=55
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=56
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=57
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=58
│   f = -4.941360193148116


┌ Info: iter=93
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=94
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=95
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=96
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=97
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=98
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=99
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=100
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=101
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=102
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=103
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=104
│   f = -4.94136019314

┌ Info: iter=141
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=142
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=143
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=144
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=145
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=146
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=147
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=148
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=149
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=150
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=151
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=152
│   f = -4.9413

┌ Info: iter=189
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=190
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=191
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=192
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=193
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=194
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=195
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=196
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8


Done. Final objective = -4.941360193148116


┌ Info: iter=197
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=198
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=199
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8
┌ Info: iter=200
│   f = -4.941360193148116
│   g∞ = 0.23748021392471363
└   Δ = 1.0e-8


In [19]:
import multiprocessing

print("CPU 总核心数:", multiprocessing.cpu_count())

LoadError: ArgumentError: Package multiprocessing not found in current path.
- Run `import Pkg; Pkg.add("multiprocessing")` to install the multiprocessing package.

In [18]:
%pip install multiprocessing

Unrecognized magic `%%pip`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.
